# Dataverse API workflow

In [9]:
import requests
import re

In [12]:
# some constants
dataverse_key = "2a301287-c8e8-43f5-9862-cf084b310341"
max_retries=5

# defining some constants
r_file_query = "fileContentType:type/x-r-syntax"

# initialize variables to store current state of scraping
page_num = 0
r_dois = []
failures = 0
numfiles = 0
print_status=True

# Get DOIs

In [13]:
myresults = requests.get("https://dataverse.harvard.edu/api/search/", 
                         params= {"q": r_file_query, "type": "file",
                                              "key": dataverse_key, "start": str(1000 * page_num),
                                              "per_page": str(1000)}).json()['data']['items']
if print_status:
    print("Parsing results from page {}...".format(page_num))

Parsing results from page 0...


# extract the DOI (if any) from the result

In [14]:
for myresult in myresults:
    doi_match = re.search("(doi:[^,]*)", myresult['dataset_persistent_id'])
    if doi_match:
        r_dois.append(doi_match.group(1) + '\n')

In [15]:
myresults[1]

{u'checksum': {u'type': u'MD5', u'value': u'4b077329d83599e1f59cc779d9477a02'},
 u'dataset_citation': u'Dolezal, Martin; Ennser-Jedenastik, Laurenz; M\xc3\xbcller, Wolfgang C.; Winkler, Anna Katharina, 2016, "Replication data for: Analyzing Manifestos in their Electoral Context: A New Approach Applied to Austria, 2002\xe2\x80\x932008", https://doi.org/10.7910/DVN/27864, Harvard Dataverse, V1',
 u'dataset_id': u'46890',
 u'dataset_name': u'Replication data for: Analyzing Manifestos in their Electoral Context: A New Approach Applied to Austria, 2002\xe2\x80\x932008',
 u'dataset_persistent_id': u'doi:10.7910/DVN/27864',
 u'description': u'',
 u'file_content_type': u'type/x-r-syntax',
 u'file_id': u'2507267',
 u'file_persistent_id': u'doi:10.7910/DVN/27864/EDKVG8',
 u'file_type': u'R Syntax',
 u'md5': u'4b077329d83599e1f59cc779d9477a02',
 u'name': u'replication appendix.r',
 u'published_at': u'2016-03-11T17:54:53Z',
 u'size_in_bytes': 26833,
 u'type': u'file',
 u'url': u'https://dataverse.

In [16]:
dois = r_dois[:2]
dois

[u'doi:10.7910/DVN/28763\n', u'doi:10.7910/DVN/27864\n']

# Query the dataverse API for all the files in a dataverse

In [7]:
for doi in dois:
    files = requests.get(
        "https://dataverse.harvard.edu/api/access/datafile/:persistentId", 
        params= {"persistentId": doi, "key": dataverse_key})
    print(files.json())

{u'status': u'ERROR', u'message': u"'/api/v1/access/datafile/:persistentId' endpoint does not exist on this server. Please check your code for typos, or consult our API guide at http://guides.dataverse.org.", u'code': 404}
{u'status': u'ERROR', u'message': u"'/api/v1/access/datafile/:persistentId' endpoint does not exist on this server. Please check your code for typos, or consult our API guide at http://guides.dataverse.org.", u'code': 404}


In [ ]:
files[1].json()['data']['latestVersion']['files']

# convert DOI into a friendly directory name by replacing slashes and colons

In [ ]:
doi_direct = storage_path + '/' + doi.replace("/", "-").replace(":", "--")

# make a new directory to store the dataset

In [ ]:
if not os.path.exists(doi_direct):   
    os.makedirs(doi_direct)

# for each file result query the API for the file contents

In [ ]:
for file in files:
    # parse the filename and fileid 
    filename = file['dataFile']['filename']
    fileid = file['dataFile']['id']
    response = requests.get("https://dataverse.harvard.edu/api/access/datafile/" + 
                            str(fileid), params={"key": dataverse_key})
    # write the response to correctly-named file in the dataset directory
    with open(doi_direct + "/" + filename, 'w') as handle:
        handle.write(response.content)